# COMP47590: Advanced Machine Learning
# Assignment 1: Implementing Perceptrons

- Student 1 Name: Carl Fabian Winkler
- Student 1 Number: 20207528
- Student 2 Name: David Moreno Boras
- Student 2 Number: 21200646

## Introduction

## Import Packages

In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn import metrics
"""
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample"""

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\n\nfrom sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state\nfrom sklearn.utils.multiclass import unique_labels\nfrom sklearn import preprocessing\n\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import resample'

## Task 1: The Perceptron Classifier
Define the PerceptronClassifier class

In [374]:
class Layer():
    def __init__(self, n_in, n_out, activation = 'Sigmoid', init = 'Xavier', output_layer = False):
        self.activation = activation
        # XW + b = y ; We input more than one sample per pass...
        
        self.output_layer = output_layer
        self.weights = np.zeros((n_in, n_out))
        self.biases = np.zeros((n_out))
        if init == 'Xavier':
            var = np.sqrt(6.0 / (n_in + n_out))
            for i in range(n_in):
                for j in range(n_out):
                      self.weights[i,j] = np.float32(np.random.uniform(-var, var))
                        
        self.d_w = np.zeros(weights.shape)
        self.d_b = np.zeros(biases.shape)
        #print("Weights:", self.weights.shape)
        #print(self.weights) 
        
    def getWeights(self):
        return self.weights

    def forward(self, x):
        """print("X:", x.shape)
        print(x)
        print("Weights:", self.weights.shape)
        print(self.weights)"""
        z = x @ self.weights + self.biases
        #print("Weights:", self.weights.shape)
        if self.activation == 'Sigmoid':
            out = 1 / (1 + np.exp(-z))
        elif self.activation == 'ReLu':
            out = np.maximum(z, 0)
        elif self.activation == 'TanH':
            out = np.tanh(z)
        else:
            out = z
        
        self.cache = (x, z)
        
        return out    
    
    def backward(self, d_out):
        inputs, z = self.cache
        weight = self.weights
        bias = self.biases
        #print("DOUT:", d_out)
        #print("shape", d_out.shape)
        if self.output_layer:
            d_act = z
        if self.activation == 'Sigmoid':
            d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))
        elif self.activation == 'ReLu':
            d_act = d_out * (z > 0)
            
        elif self.activation == 'TanH':
            d_act = d_out * np.square(z)
        else:
            d_act = z
        #print("d_act:", d_act)
        #print("shape", d_act.shape)    
        d_inputs = d_act @ weight.T
        self.d_w = inputs.T @ d_act
        self.d_b = d_act.sum(axis=0) 
        
        return d_inputs#, self.d_w, self.d_b
    
    def update_gd_params(self, lr):
        #print("Weight upd: ", self.d_w)
        self.weights = self.weights - lr * self.d_w
        self.biases = self.biases - lr * self.d_b

class PerceptronClassifier(BaseEstimator, ClassifierMixin):
    
    """
    Parameters
    ----------
    Attributes
    ----------
    Notes
    -----
    See also
    --------
    Examples
    --------
    """
    # Constructor for the classifier object
    def __init__(self, in_dim, out_dim, hidden_units, n_layers, activation = 'Sigmoid', 
                 learning_rate = 0.01, weight_decay = 0, epochs = 30, regularisation = 'L2'):

        """Setup a Perceptron classifier .
        Parameters
        ----------
        Returns
        -------
        """     
        self.epochs = epochs
        self.layers = []
        self.lr = learning_rate
        self.regularisation = regularisation
        
        self.layers.append(Layer(in_dim, hidden_units, activation, 'Xavier'))
        for l in range(n_layers):
            self.layers.append(Layer(hidden_units, hidden_units, activation, 'Xavier'))
        self.layers.append(Layer(hidden_units, out_dim, activation, 'Xavier', output_layer = True))
        
        print("Layers:", len(self.layers))
        
        # Initialise class variabels
    def forward(self, X):
        out = self.layers[0].forward(X)
        for layer in self.layers[1:]:
            out = layer.forward(out)
        return out
                
    def backward(self, in_grad):
        i = len(self.layers) - 2 
        # d_inputs, _, _ = lay.backward(in_grad)
        next_grad = self.layers[i+1].backward(in_grad)
        while i >= 0:
            next_grad = self.layers[i].backward(next_grad)
            i -= 1 
            
    def loss_back(self, ground, pred):
        l2_loss = np.sum(np.power(ground-np.squeeze(pred, axis=1), 2)) / len(ground)
        upstream_grad = -np.expand_dims(ground-np.squeeze(pred),axis=1)
        #print("Mean Loss:", l2_loss)
        return l2_loss, upstream_grad
            
    def l2_loss_back(self, y_hat, pred):
        return -np.expand_dims(y_hat-np.squeeze(pred),axis=1)
        
    # The fit function to train a classifier
    def fit(self, X, y):
        # WRITE CODE HERE
        
        loss_curve = []
        
        for i in range(self.epochs):
            out = self.forward(X)
            #print("Prediction:",out)
            
            l2_loss, grad = self.loss_back(y, out)
            loss_curve.append(l2_loss)
            # Backpropagation
            self.backward(grad)
            
            # Update weights and biases
            for layer in self.layers:
                layer.update_gd_params(self.lr)
        return
    
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        return self.forward(X)
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        tmp = self.forward(X)
        sum1 = tmp.sum(axis = 1)
        out = X.T / sum1
        out = out.T
        return out

## Task 2: Evaluation

#### Load the Diabethic Retinopathy dataset

In [384]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

diabetic_af = pd.read_csv('messidor_features.csv', na_values='?', index_col = 0)
diabetic_af.head()
y = diabetic_af.pop('Class').values
x_raw = diabetic_af.values
x_norm = NormalizeData(x_raw)
y_numbers = list([int(x[2]) for x in y])
y_numbers = np.array(y_numbers)

x_train, x_test, y_train, y_test = train_test_split(x_norm, y_numbers, shuffle=False, train_size = 0.6)

x_train = NormalizeData(x_train)
x_test = NormalizeData(x_test)
y_train = NormalizeData(y_train)
y_test = NormalizeData(y_test)

#### Check the model
To make sure that our model is abled to learn and that it works properly we overfit on a very small part of the data. In this case we reach an accuracy of 100 %. The model therefore clearly has the ability to learn and the capacity to learn and even ovefit the data.

In [398]:
x_overfit = x_train[0:15]
y_overfit = y_train[0:15]

clf = PerceptronClassifier(len(x_train[0]), 1, 30, 1, learning_rate=0.5, epochs=10000)
clf.fit(x_overfit, y_overfit)
y_pred = clf.predict(x_overfit)
y_pred_binary = [1 if x >= 0.5 else 0 for x in y_pred]
accuracy = metrics.accuracy_score(y_pred_binary, y_overfit)
print('Accuracy of the overfitting is: ', accuracy)

Layers: 3
Accuracy of the overfitting is:  1.0


#### Create a Baseline for the data
To have a model that the perceptron can be compared to a baseline model is created here. Therefore a perceptron model of the sklearn library is trained to the data. We are trying to reach or even be better than this baseline model in terms of accuracy with the perceptron implemented here. 

In [238]:
#(self, in_dim, out_dim, hidden_units, n_layers,
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])
clf = PerceptronClassifier(2, 1, 5, 0, regularisation='None', learning_rate = 10,activation='Sigmoid')
clf.fit(x, y,epochs=1000)

Layers: 2


## Task 3 & 4: Add Different Activations & Regularisation

Reimplement the PerceptronClassifier class adding an activation function option and L2 regularisation. 

In [ ]:
class PerceptronClassifier2(BaseEstimator, ClassifierMixin):
    """
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------

        
        Returns
        -------

        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels

        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):

        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        
        # WRITE CODE HERE

Load the dataset and explore it.

## Task 5: Reflect on the Performance of the Different Models Evaluated

Perform hyper-parameter tuning and evaluate models. 

## Test the Diabetic Retiniphaty dataset

In [239]:
diabetic_af = pd.read_csv('messidor_features.csv', na_values='?', index_col = 0)
diabetic_af.head()
y = diabetic_af.pop('Class').values
x_raw = diabetic_af.values
print("Features: ", x_raw[0:2])
print("Class: ", y[0:10])

Features:  [[1.0000000e+00 2.2000000e+01 2.2000000e+01 2.2000000e+01 1.9000000e+01
  1.8000000e+01 1.4000000e+01 4.9895756e+01 1.7775994e+01 5.2709200e+00
  7.7176100e-01 1.8632000e-02 6.8640000e-03 3.9230000e-03 3.9230000e-03
  4.8690300e-01 1.0002500e-01 1.0000000e+00]
 [1.0000000e+00 2.4000000e+01 2.4000000e+01 2.2000000e+01 1.8000000e+01
  1.6000000e+01 1.3000000e+01 5.7709936e+01 2.3799994e+01 3.3254230e+00
  2.3418500e-01 3.9030000e-03 3.9030000e-03 3.9030000e-03 3.9030000e-03
  5.2090800e-01 1.4441400e-01 0.0000000e+00]]
Class:  ["b'0'" "b'0'" "b'1'" "b'0'" "b'1'" "b'1'" "b'1'" "b'0'" "b'1'" "b'1'"]


### Train and predict using our classifier

With a single split

In [170]:
y_numbers = list([int(x[2]) for x in y])
y_numbers = np.array(y_numbers)
y_numbers

array([0, 0, 1, ..., 0, 1, 0])

In [179]:
print(x_train.shape)
print(y_train.shape)

(11, 18)
(11,)


In [173]:
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])
print(x.shape)
print(y.shape)

(4, 2)
(4,)


In [232]:
y_train.shape

(11,)

In [234]:
x_train.shape

(11, 18)

In [ ]:
in_dim, out_dim, hidden_units, n_layers, activation

In [259]:
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_numbers, shuffle=True, train_size = 0.1)

clf = PerceptronClassifier(len(x_train[0]), 1, 35, 0, learning_rate = 0.03)
clf.fit(x_train, y_train, epochs=100)
y_pred = clf.predict(x_test)


Layers: 2


In [258]:
y_pred

array([[0.22727399],
       [0.22726872],
       [0.22730339],
       ...,
       [0.22726881],
       [0.22726882],
       [0.78511663]])

### Grid search

Do grid search with the train set, use the test set for evaluation

In [ ]:
cv_folds = 5
param_grid ={'activation': ['Sigmoid', 'ReLu', 'TanH'], 'regularisation':['None', 'L2']}

# Perform the search
tuned_perceptron = GridSearchCV(PerceptronClassifier(), \
                            param_grid, cv=cv_folds, verbose = 2, \
                            n_jobs = -1)
cross_val_score(clf, x_train, y_train, cv=10)